In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests


C:\Users\disha.patel\AppData\Local\Continuum\Miniconda3\envs\carnd-term1\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))


TensorFlow Version: 1.5.0


C:\Users\disha.patel\AppData\Local\Continuum\Miniconda3\envs\carnd-term1\lib\site-packages\ipykernel_launcher.py:7: UserWarning: No GPU found. Please use a GPU to train your neural network.
  import sys


In [3]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()
    w1 = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    l3 = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    l4 = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    l7 = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    
    return w1, keep, l3, l4, l7
tests.test_load_vgg(load_vgg, tf)

Tests Passed


In [15]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    conv_1x1 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, padding='same',
                                kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    
    conv_4 = tf.layers.conv2d(vgg_layer4_out, num_classes, 1, padding='same',
                             kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    conv_3 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, padding='same',
                             kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    deconv_32 = tf.layers.conv2d_transpose(conv_1x1, num_classes, 4, strides=(2, 2), padding='same',
                                          kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    
    add_skip1 = tf.add(deconv_32, conv_4)
    
    deconv_16 = tf.layers.conv2d_transpose(add_skip1, num_classes, 4, strides=(2,2), padding='same',
                                          kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    
    add_skip2 = tf.add(deconv_16, conv_3)
    output = tf.layers.conv2d_transpose(add_skip2, num_classes, 32, strides=(8,8), padding='same', 
                                        kernel_regularizer=tf.contrib.layers.l2_regularizer(1e-3))
    
    
    
    #tf.Print(output, [tf.shape(output)[1:3]])
    
    
    return output
tests.test_layers(layers)

In [16]:

def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    logits = tf.reshape(nn_last_layer, (-1, num_classes))
    correct_label = tf.reshape(correct_label, (-1, num_classes))
    
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
                        logits=logits, labels=correct_label
                    )
    mean_cross_entropy = tf.reduce_mean(cross_entropy)
    opt = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = opt.minimize(mean_cross_entropy, name="training_op")
   

    
    #return None, None, None
    
    return logits, training_op, mean_cross_entropy
tests.test_optimize(optimize)

In [10]:
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    for epochs in range(epochs):
        for image, label in get_batches_fn(batch_size):
            # Training 
            pass
            _,loss = sess.run([train_op, cross_entropy_loss], 
                             feed_dict={input_image: image, correct_label: label, 
                                       keep_prob: 0.8, learning_rate: 1e-4})

            print("Epoch {}".format(epochs + 1), " loss: {:.4f}".format(loss))

        '''loss, _ = sess.run([cross_entropy_loss, train_op], feed_dict={
                input_image: X_batch,
                correct_label: y_batch,
                keep_prob: KEEP_PROB, // can be between 0 and 1 during training
                learning_rate: LEARNING_RATE
            })'''

        
    pass
tests.test_train_nn(train_nn)

In [13]:

def run():
    num_classes = 2
    image_shape = (160, 576)
    data_dir = './data'
    runs_dir = './runs'
    tests.test_for_kitti_dataset(data_dir)

    # Download pretrained vgg model
    helper.maybe_download_pretrained_vgg(data_dir)

    # OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
    # You'll need a GPU with at least 10 teraFLOPS to train on.
    #  https://www.cityscapes-dataset.com/

    with tf.Session() as sess:
        # Path to vgg model
        vgg_path = os.path.join(data_dir, 'vgg')
        # Create function to get batches
        get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

        # OPTIONAL: Augment Images for better results
        #  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network

        # TODO: Build NN using load_vgg, layers, and optimize function
        input_image, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)
        layer_output = layers(layer3_out, layer4_out, layer7_out, num_classes)
        
        label = tf.placeholder(tf.int32, shape=[None, None, None, num_classes])
        learning_rate = tf.placeholder(tf.float32)
        logits, train_op, loss = optimize(layer_output, label, learning_rate, num_classes)
        # TODO: Train NN using the train_nn function
        saver = tf.train.Saver()

        # TODO: Save inference data using helper.save_inference_samples
        #  helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)
        sess.run(tf.global_variables_initializer())
        train_nn(sess, epochs, batches, get_batches_fn, train_op, loss, 
                input_image, label, keep_prob, learning_rate)
        
        helper.save_inference_samples(runs_dir, data_dir, sess, image_shape, logits, keep_prob, input_image)
        
        saver.save(sess, './runs/sem_seg_model.ckpt')

        # OPTIONAL: Apply the trained model to a video
if __name__ == '__main__':
    # pass
    run()

997MB [11:57, 1.39MB/s]                                                                                                


NameError: name 'epochs' is not defined